Я снова сделал всё дз в отдельном python-файлике, потому что его гораздо удобнее много раз запускать и дебажить

Перенесу в ноутбук самые интересные функции, остальные были в предыдущих дз

In [1]:
import dill
import numpy as np
import scipy
import scipy.stats as st
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score

from cips import CIPS


def prob_x_ge_y(m1, d1, m2, d2):
    return 1 - st.norm.cdf(0, m1 - m2, (d1 ** 2 + d2 ** 2) ** 0.5 + 1e-6)


def run(lambda_=10, saved=True):
    # датасет, датасет на последнем дне, датасет с измененнм banner_id, датасет с измененнм banner_id на последнем дне, информация о баннерах, информация о баннерах на последнем дне, столбец с кликами, столбец с кликами на последнем дне
    X, X_last, X_1, X_1_last, X_mod, X_mod_last, y, y_last = CIPS().get_data(nrows=None)  # None = все строчки

    # модель долго учится, сохраним в файл чтобы внизу удобно было дебажить CIPS
    if not saved:
        model = CIPS().get_model()
        model.fit(X, y)
        with open('model.ckpt', 'wb') as f:
            dill.dump(model, f)
    else:
        with open('model.ckpt', 'rb') as f:
            model = dill.load(f)

    # считаем pi_0
    pi0_probs = prob_x_ge_y(X_mod_last['coeff_sum0'],
                            X_mod_last['g0'],
                            X_mod_last['coeff_sum1'],
                            X_mod_last['g1'])

    # средние для pi_1
    pi1_coeff_sum_new_0 = scipy.special.logit(model.predict_proba(X_last)[:, 1])
    pi1_coeff_sum_new_1 = scipy.special.logit(model.predict_proba(X_1_last)[:, 1])

    # считаем pi_1
    pi1_probs = prob_x_ge_y(pi1_coeff_sum_new_0,
                            X_mod_last['g0'],
                            pi1_coeff_sum_new_1,
                            X_mod_last['g1'])

    # оставляем конечные значения, не слишком маленькие
    mask = np.isfinite(pi1_probs) * np.isfinite(pi0_probs) * (np.abs(pi0_probs) > 1e-18)

    # посчитали cips
    cips = np.mean(y_last[mask] * np.minimum(lambda_, (pi1_probs[mask] / pi0_probs[mask])))

    # посчитаем ещё настоящее среднее (которое хотим оценивать cips-ом)
    true_value = np.mean(y_last)

    # вдогонку, посчитаем auc и log loss
    auc = roc_auc_score(y_last, model.predict_proba(X_last)[:, 1])
    ll = log_loss(y_last, model.predict_proba(X_last)[:, 1])
    print(f'Auc: {auc}, ll: {ll}')

    return cips, true_value

In [2]:
ans = run()
print(f'CIPS = {ans[0]}')
print(f'True mean reward = {ans[1]}')

Auc: 0.781078976974208, ll: 0.13737523960121442
CIPS = 0.05716803453417325
True mean reward = 0.03624213329158208


Выкладки о сравнении двух нормальных величин:

$$X \sim N(m_1, d_1^2)$$

$$Y \sim N(m_2, d_2^2)$$


$$P(X > Y) = P(X - Y > 0) = 1 - P(X - Y < 0) = 1 - F_{X-Y}(0)$$

$X-Y$ - разность двух нормальных случайных величин, значит и сама величина нормальная. Матожидание - разность матожиданий, дисперсия - сумма дисперсий

$$X-Y \sim N(m_1 - m_2, d_1^2 + d_2^2)$$

Так как дисперсия может быть очень маленькой, стоит добавлять к ней 1e-6 или что-то похожее